In [ ]:
# https://github.com/gcdart/MulticlassClassifier/blob/master/src/ml/LogisticRegression.java
# https://www.kaggle.com/c/lshtc/discussion/6911#38233 - preprocessing: multilabels comma should not have spaces
# https://www.kaggle.com/c/lshtc/discussion/14048 - dataset statistics
## reading the LWIKI, SWIKI dataset

In [ ]:
import sys

In [ ]:
sys.version

In [1]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.datasets import fetch_rcv1

In [2]:
from joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

In [3]:
@mem.cache
def get_data(filename):
    
    try:
        data = load_svmlight_file(filename, multilabel=True)
    
    except:
        # when there is an error since the dataset isn't in the correct libsvm format. *sigh*
        # so we need to ValueError: Feature indices in SVMlight/LibSVM data file should be sorted and unique get rid of that
        # *SIGH*
        
        # *call function to preprocess*
    return data[0], data[1]

In [52]:
from collections import OrderedDict

In [53]:
file = open("swiki/data/train_remapped.txt", "w")

In [55]:
with open("swiki/data/train.txt", "r") as f:
    the_file = ''
    for i, line in enumerate(tqdm(f)):
        instance = line.strip().split()
        labels = instance[0]
        doc_dict = OrderedDict()
        temp_dict = {}
        temp_string = ''
        
        for pair in instance[1:]:
            feat = pair.split(":")
            if int(feat[0]) not in temp_dict:
                temp_dict[int(feat[0])] = int(feat[1])
        
        for key in sorted(temp_dict.keys()):
            doc_dict[key] = temp_dict[key]
        
        for feat, tf in doc_dict.items():
            temp_string = temp_string + "{}:{} ".format(feat, tf)        
        temp_string = labels + " " + temp_string
        the_file = the_file + temp_string + "\n"

57767it [06:16, 68.20it/s] 

KeyboardInterrupt: 

In [56]:
file.write(the_file)
file.close()

57767it [06:30, 68.20it/s]

In [4]:
# x contains all the feat:value data
# y contains all the labels (per line)
x, y = get_data("swiki/data/train.txt")
# a, b = get_data("test-remapped/test-remapped.csv")

________________________________________________________________________________
[Memory] Calling __main__-C%3A-users-harshasivajit-Documents-master-ai-rr13-__ipython-input__.get_data...
get_data('swiki/data/train.txt')


ValueError: Feature indices in SVMlight/LibSVM data file should be sorted and unique.

In [ ]:
x.shape #num documents X num features (training set)

In [ ]:
print("{} size".format(x.shape))

In [ ]:
len(y)

In [ ]:
# lets see how many unique labels are there and if it verifies with the paper
# ooooh i only get the leaf labels with this. or like atleast when i add it to a set.
# how do i get class labels? what are class labels first!
leaf_labels = set()
class_labels = []
count_y = []
for i in y:
    count_y.append(len(i))
    for j in i:
        leaf_labels.add(j)
        class_labels.append(j)

In [ ]:
print(len(leaf_labels))

In [ ]:
# these leaf labels range from 1 - 445729 (training set) this number is the max label id
list(leaf_labels)[325055] #[0]

In [ ]:
# this is just the plain brute force counting of all labels for all documents.
# same as count_y
len((class_labels))

In [ ]:
# avg num_labels per instance(training doc)
sum(count_y)/len(y)

### test set

In [ ]:
a.shape
# len(b)

In [ ]:
leaf_t = set()
class_t = []

for i in b:
    for j in i:
        leaf_t.add(j)
        class_t.append(j)

In [ ]:
len(class_t)

In [ ]:
904334/2 # <- num of labels are doubled basically due to the 0 in every document

### hierarchy.txt

In [ ]:
def read_hier(filename):
    
    unique_h = set()
    
    with open(filename, "r") as f:
        for i, line in enumerate(f):
            words = line.strip().split()
            for w in words:
                unique_h.add(int(w))

    return unique_h

In [ ]:
u = read_hier("hierarchy/hierarchy.txt")

In [ ]:
len(u)

In [ ]:
max(u)